In [4]:
import plotly.graph_objects as go
import networkx as nx
import numpy as np

from utils import (get_edges, 
                   get_nodes, 
                   get_degree, 
                   build_all_to_all, 
                   build_graph_viz, 
                   build_node_trace, 
                   build_edge_trace)

# Cooperative and Competitive Homogeneous Agents

In [7]:
G = build_all_to_all(10)

In [8]:
adjacency_matrix = nx.linalg.graphmatrix.adjacency_matrix(G)
print(adjacency_matrix.todense())

[[0 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 0 1 1]
 [1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 0]]


In [7]:
# resistance term: d_{ij} 
# In the general model, a larger dij implies a greater resistance of agent i to 
# forming a non-neutral opinion about option j

# social term: 
# the  product  of  an attention parameter ui ≥ 0 
# and a saturating function of weighted sums of agent opinions that are available 
# to agent i and influence its opinion  of  option j
# The  social  term  can  also  be  interpreted as an activation term

# The magnitude of Ajl_ik determines the  strength  of  influence  of  agent k’s  
# opinion  about  option l on  agent i’s  opinion  about  option j,  and  the
# sign  of Ajl_ik determines whether this interaction is excitatory (Ajlik>0) 
# or inhibitory (Ajlik<0).

# The input bij ∈ R represents  an  input  signal  from  the environment  
# or  a  bias  or  predisposition  that  directly  affects agent i’s  opinion  of  option j

In [8]:
# eq 5a (z is the relative opinion)
# z_ij_dot = F_ij(Z) - 1/N_o \sum_l F_il(Z)
# F_ij(Z) = -d_ij z_ij + u_i ( S_1(\sum_k Ajj_ik z_kj) + \sum_(l neq j) S_2(\sum_k Ajl_ik z_kl)) + b_ij

In [184]:
def compute_zdot(D, u, Z, A, B):
    '''
    computes equation of motion of relative opinions
    
    Zij_dot = F_ij(Z) - (1/No) * \sum_(l = 1)^No F_il(Z)
    
    Arguments:
    ---------
    D: matrix 
        size - (number of agents x number of options)
        resistance term. In the general model, a larger dij implies a greater resistance of agent i to 
        forming a non-neutral opinion about option j.
    
    u: vector
        len(U) = number of agents
        attention parameter.
        
    Z: matrix
        size - (number of agents x number of options)
        
    A: tuple
        (Intra-agent, same-option coupling: Ajj_ii (not needed)
         Intra-agent, inter-option coupling: Ajl_ii, j neq l (not needed)
         Inter-agent, same-option coupling: Ajj_ik, i neq k (Size: No x Na x Na)
         Inter-agent, inter-option coupling: Ajl_ik, i neq k,j neq l (Size: (No-1) x No x Na x Na) )
        
         
    B: matrix
    '''
    
    No = len(A[0])
    
    F = compute_drift(D, u, Z, A, B)
    
    average_option_drift = 1/No * np.einsum('il->i', F)

    return F - average_option_drift[:, np.newaxis]

In [185]:
def compute_drift(D, u, Z, A, B):
    '''
    computes drift using S_1 = S_2 = tanh
    
    F_ij(Z) = -d_ij z_ij + u_i ( S_1(\sum_k Ajj_ik z_kj) + \sum_(l neq j) S_2(\sum_k Ajl_ik z_kl)) + b_ij
    
    
    Arguments:
    ---------
    D: matrix 
        size - (number of agents x number of options)
        resistance term. In the general model, a larger dij implies a greater resistance of agent i to 
        forming a non-neutral opinion about option j.
    
    u: vector
        len(U) = number of agents
        attention parameter.
        
    Z: matrix
        size - (number of agents x number of options)
        
    A: tuple
        (Intra-agent, same-option coupling: Ajj_ii (not needed)
         Intra-agent, inter-option coupling: Ajl_ii, j neq l (not needed)
         Inter-agent, same-option coupling: Ajj_ik, i neq k (Size: No x Na x Na)
         Inter-agent, inter-option coupling: Ajl_ik, i neq k,j neq l (Size: (No-1) x No x Na x Na) )
        
         
    B: matrix
    '''
#     intra_agent_same_option = A[0] # 2D matrix
#     intra_agent_inter_option = A[2]
#     inter_agent_same_option = A[3] # 3D matrix
#     inter_agent_inter_option = A[4]

    # Inter-agent matrices include intra-agent coupling to match summations in paper
    inter_agent_same_option = A[0] # 2D matrix
    inter_agent_inter_option = A[1] # 3D matrix
    
    social_term = compute_social_term(inter_agent_same_option, inter_agent_inter_option, Z, u)
    
    # F_ij(Z) = -d_ij z_ij + u_i ( S_1(\sum_k Ajj_ik z_kj) + \sum_(l neq j) S_2(\sum_k Ajl_ik z_kl)) + b_ij
    return -D * Z + social_term + B

In [186]:
def compute_social_term(inter_agent_same_option, inter_agent_inter_option, Z, u):
    # social term is
    # the  product  of  an attention parameter ui ≥ 0 
    # and a saturating function of weighted sums of agent opinions that are available 
    # to agent i and influence its opinion  of  option j
    
    weighted_inter_agent_same_option = np.einsum('jik,kj->ij', inter_agent_same_option, Z)
    thresholded_weighted_inter_agent_same_option = np.tanh(weighted_inter_agent_same_option)
    
    weighted_inter_agent_inter_option = np.einsum('jlik,kl->ijl', inter_agent_inter_option, Z)
    thresh_weighted_inter_agent_inter_option = np.tanh(weighted_inter_agent_inter_option)

    # avg_thresh_weighted_inter_agent_inter_option = np.mean(thresh_weighted_inter_agent_inter_option, axis=1)
    
    # Sum the activations over each distinct option l (check: should be summing over No-1 options)
    cumulative_thresh_weighted_inter_agent_inter_option = \
        np.einsum('lij->ij', thresh_weighted_inter_agent_inter_option)
    
    social_term = u * (thresholded_weighted_inter_agent_same_option + \
                       cumulative_thresh_weighted_inter_agent_inter_option)
    
    return social_term

In [187]:
def test_compute_drift():
    
    Na = 3 # number of agents
    No = 4 # number of options
    
    # For each option, we get an activation for all Na agents from each other agent (including self-activations)
    inter_agent_same_option = np.ones((No, Na, Na)) 
    
    # For each option, for each agent, we get an activation from all Na agents 
    # about each (No-1) other distinct options
    inter_agent_inter_option = np.ones((No-1, No, Na, Na)) 
    
    # Opinion matrix of agent i, opinion j
    Z = np.random.rand(Na, No)
    
    # Drift Coefficient Matrix
    D = np.random.rand(Na, No)
    
    # Bias Matrix
    B = np.random.rand(Na, No)
    
    # Attention parameter
    u = np.random.rand(Na,1)

    # Make the tuple
    A = (inter_agent_same_option, inter_agent_inter_option)
    
    compute_social_term(inter_agent_same_option, inter_agent_inter_option, Z, u) 
    
    F = compute_drift(D, u, Z, A, B)
    
    #print(F)
    
    Zd = compute_zdot(D, u, Z, A, B)
    
    print(Zd)


In [188]:
# DRIVER

test_compute_drift()

[[-0.27901138  0.32894165 -0.40482712  0.35489685]
 [-0.38796684  0.36452905  0.37626008 -0.35282229]
 [-0.24467308  0.21755359 -0.51949277  0.54661227]]
